# Source: Rivals

In [1]:
#hide
import core_constants as cc
import functions as fx
import json

## Set Notebook Settings

In [2]:
conference = 'american'

years = cc.get_defYears()
headers= cc.get_header()
schoolsList = cc.get_schoolsList()
teamDirectory = cc.get_htmlDir('rivals', conference, 'teams')
playerDirectory = cc.get_htmlDir('rivals', conference, 'recruits')
#testDirectory = '..//tests//'

## Get & Save the Teams & Players Page HTML
#### Source: https://maryland.rivals.com/commitments/football/2012
> This page contains metadata of each player along with the Rivals ranking and stars.  Unlike 247Sports, we process the fetch and save of both pages directly from a single function

In [ ]:
fx.get_Rivals(conference, schoolsList, years, headers, sleepyTime=6)

## Process Local Rivals HTML Files

> All of this processing is done locally, using the files saved in the previous few steps.  This creates an exhaustive store of all the fields grabbed from the scrapes.

In [3]:
cc.save_records('scrapedData', 'rivals_' + conference, fx.process_Rivals(playerDirectory, conference, schoolsList))

## Summarize the Dataset

> We don't need repetitive fields across the various datasets (ie - I don't need height coming back from 3 sources).  This step strips to only what I care about for the master print out.

In [3]:
outputDir = '..//summarizedData//'
dataset = 'rivals'

with open(outputDir + dataset + "data.json", "w", encoding="utf-8") as write_file:
                write_file.write(json.dumps(fx.summarize_Rivals()))